In [1]:
import os; os.chdir('../..') # changing to root directory of project
from config import config

In [2]:
import pandas as pd 
import pickle
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from utils.save_features import save_features

In [3]:
df = pd.read_csv("data/processed/stroke.csv")

In [4]:
# Feature selecting
features = df.drop(['stroke'], axis = 1).columns
features

Index(['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status'],
      dtype='object')

In [5]:
# saving features in json file
save_features(df[features], 'stroke')

Starting to process columns in feature_df...
features config file doesnt exist creating new
Updating features config file
Succesfully updated features config file


In [6]:
# encoding categorical datas
# initializing encoder
encoder = OneHotEncoder(sparse_output = False)

# columns to encode
categorical_columns = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']

# training encoder 
encoder.fit(df[categorical_columns])

# encoding
encoded_columns = pd.DataFrame(encoder.transform(df[categorical_columns]))

# naming columns
encoded_columns.columns = encoder.get_feature_names_out(categorical_columns)

# concating with original dataframe
df = pd.concat([df.drop(categorical_columns, axis = 1), encoded_columns], axis = 1)

# saving encoder to encode data from user
with open('models/stroke/encoder.pkl', 'wb') as fp: 
    pickle.dump(encoder, fp)

In [7]:
# Normalizing numerical features 
# initializing standared scaler
scaler = StandardScaler()

# features to scale
numerical_features = ['age', 'avg_glucose_level', 'bmi']

# training scaler
scaler.fit(df[numerical_features])

# scaling
df[numerical_features] = scaler.transform(df[numerical_features])

# saving model 
with open('models/stroke/scaler.pkl', 'wb') as fp:
    pickle.dump(scaler, fp)

In [9]:
# spliting train test data
X = df.drop(['stroke'], axis = 1) # features
y = df['stroke'] # target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [10]:
# saving train data
X_train.to_csv('data/model_training/stroke/X_train.csv', index=False)
X_test.to_csv('data/model_training/stroke/X_test.csv', index=False)
y_train.to_csv('data/model_training/stroke/y_train.csv', index=False)
y_test.to_csv('data/model_training/stroke/y_test.csv', index=False)